In [2]:
import numpy as np
import math 
import random
from numpy import linalg 
from numpy import real
from scipy import linalg, matrix
from sympy import Matrix
A={0:(0,1), 1:(2,3), 2:(0,2), 3:(0,3), 4:(1,2), 5:(1,3)}

kex=3
kin=0.5

def distance(X1,X2):
	Sum=0.0
	for i in range(len(X1)):
		Sum=Sum+(X1[i]-X2[i])*(X1[i]-X2[i])
	return math.sqrt(Sum/len(X1))

### Sorting things ### 
def getKey(item):
    return item[0]


### Generating all sets of states, 00, 01, 10, 11  ### 
def OrderSets():
#	List=[0]*6
	Allset=[]
	List=[]
#	Allset.append(List)
#	print Allset
	NewSet=[]
	Set(Allset,List,0)
	for i in range(len(Allset)):
#		print list(reversed(Allset[i]))
		NewSet.append(Allset[i])
#		print Allset[i]
#		NewSet.append(list(reversed(Allset[i])))
#	print list(reversed(NewSet))
	return NewSet
#	return NewSet



## Generating all sets of states 00, 10, 01, 11 ##
def AllSets():
#	List=[0]*6
	Allset=[]
	List=[]
#	Allset.append(List)
#	print Allset
	NewSet=[]
	Set(Allset,List,0)
	for i in range(len(Allset)):
#		print list(reversed(Allset[i]))
#		print Allset[i]
		NewSet.append(list(reversed(Allset[i])))
#	print list(reversed(NewSet))
	return NewSet

def Set(Allset,List,index):
#	print index
	if len(List)==6:
		Allset.append(List)
	else:
		for i in range(2):
			Set(Allset,List+[i],index+1)
## Indexing the states ## 

## Indexing the states ## 
def state(N1):
	N=list(reversed(N1))
	L=2**N[0]
	for i in range(1,len(N)):
		L=L+N[i]*(2**((i)*N[i]))
	return L-1



## Measuring transition probability ##
def stay(N1,k):
	L1=state(N1)
	prob=0
	if k[0]==kin:
		prob=prob+1
	if k[1]==kin:
		prob=prob+1
	if k[2]==kex:
		prob=prob+1
	if k[3]==kex:
		prob=prob+1
	prob=float(format(prob,'.5f'))
	if prob==0:
		prob=0
	return (L1,prob)

def Tran(N1,N2,k):
    prob=0
    L1=state(N1)
    L2=state(N2)

    index=[]
    for i in range(len(N1)):
        if N1[i]!=N2[i]:
            prob=0
            if N1[i]==0:
                if i in range(2,6):
                    (x,y)=A[i]
                    prob=0
                    if k[y]!=3:
                        prob=prob+1.0/(3-k[y])
                    if k[x]==0:
                        prob=prob+1.0/(3-k[x])

                if i==0:
                    prob=0
                    (x,y)=A[i]
                    if k[x]==0:
                        prob=prob+1.0/(3-k[x])
                    if k[y]==0:
                        prob=prob+1.0/(3-k[y])
                if i==1:
                    (x,y)=A[i]
                    prob=0
                    if k[x]!=3:
                        prob=prob+1.0/(3-k[x])
                    if k[y]!=3:
                        prob=prob+1.0/(3-k[y])

            else:
                if i in range(2,6):
                    (x,y)=A[i]
                    if k[x]!=0:
                        prob=1.0/(k[x])

                if i==0:
                    prob=0
                    (x,y)=A[i]
                    if k[x]!=0:
                        prob=prob+1.0/(k[x])
                    if k[y]!=0:
                        prob=prob+1.0/(k[y])

                if i==1:
                    prob=0
    prob=float(format(prob,'.5f'))
    if prob==0:
        prob=0
    return (L2,prob)




## returning W matrix ## 

def WMatrix():
	W=np.zeros((64, 64))
	New=OrderSets()
	for i in New:
		Nb(i, W)
	return W 


## Printing W Matrix in Latex Format ## 

def printZeroIndex(W, size):
	s=''
	for i in range(0,size):
		for j in range(0,size):
                        W[i,j]=float(format(W[i,j],'.10f'))
                        
#			print W[i,j]
			if W[i,j]==1.0:
#				print W[i][j]
				wxy="1"						
			elif W[i,j]==0.0:
				wxy="0"			
			elif W[i,j]==0.33:
				wxy="\\frac{1}{3}"
			elif W[i,j]==1.5:
				wxy="\\frac{3}{2}"
			elif W[i,j]==2.0:
				wxy="2"
			elif W[i,j]==0.83:
				wxy="\\frac{5}{6}"				
			elif W[i,j]==0.67:
				wxy="\\frac{2}{3}"
			elif W[i,j]==1.33:
				wxy="\\frac{4}{3}"
			elif W[i,j]==0.5:
				wxy="\\frac{1}{2}"
			else:
				wxy=str(W[i,j])
			if j!=size-1:
				s=s+wxy+"&"								
			else:
				s=s+wxy+" \\"+"\\"+"\n"
	print s


'''
			if W[i][j]==0.0:
				wxy="0"			
			elif W[i][j]==0.33:
				wxy=" \\frac{1}{3} "
			elif W[i][j]==1.33:
				wxy=" \\frac{4}{3} "
			elif W[i][j]==0.5:
				wxy=" \\frac{1}{2} "
			else:
'''				


## Measuring the row related state N1 in W matrix ## 
def Nb(N1, W):
    L1=state(N1)
    Neig=[]
    k=[0]*4
#	Neig.append(N1)
    for i in range(len(N1)):
        Neig.append(N1[:i]+[1-N1[i]]+N1[i+1:])
#    print Neig
    k[0]=N1[0]+N1[2]+N1[3]
    k[1]=N1[0]+N1[4]+N1[5]
    k[2]=N1[2]+N1[4]+N1[1]
    k[3]=N1[3]+N1[5]+N1[1]

    Grand=[]
    Grand.append(stay(N1,k))
    for i in Neig:
        Z=Tran(N1,i,k)
        Grand.append(Z)
    Grand1=sorted(Grand,key=getKey)	
    s=''
    for i in range(len(Grand1)-1):
        (x,y)=Grand1[i]
        W[L1][x]=y
        s=s+'('+str(L1)+','+str(x)+') = '+str(y)+'&'
    (x,y)=Grand1[len(Grand1)-1]
    W[L1][x]=y
#    print y
    s=s+'('+str(L1)+','+str(x)+') = '+str(y)+" \\"	
    s=s+"\\"
#	print s

    s1="Transition from state " +str(L1)+", "+"$"+str(N1)+"$:"
#	print s1

    s2="\\begin{equation} \\nonumber \n"+"W("+str(L1)+",:)=c\n"+"\\begin{pmatrix}\n"
    s3=s+"\n"+"\end{pmatrix} \n"+"\end{equation} \n"
    S=s1+"\n"+s2+s3



#print OrderSets()

#print AllSets()
W= WMatrix()

print W[16,19]
print W[19,16]
#AllSets()
#print W
B=np.matrix(W)
A=B.T
#print A[16,16]
#printZeroIndex(A[16:32,16:32],16)
#for i in range(16,32):
#	for j in range(16,32):
#		print(i-15,j-15,A[i,j])
#	print "####"

C=A
#A = np.matrix([[0.25,0.25,0.5] , [0.1,0.8,0.1] , [0.6,0.3,0.1]])
#B = np.matrix([[0.25,0.25,0.5] , [0.1,0.8,0.1] , [0.6,0.3,0.1]])
#print B
X0 = np.random.random((64,1))
#X0=np.matrix(X0)
iterations = 20000
#print X0
for i in range (iterations):
	y = A*X0
#	print y
#	print max(y)
	X1=y/max(y)
	A = A*A		
	if distance(X1,X0) < 10**-8:
#		print i
		break
	X0=X1	

#print max(y)
#print X0
Sum=sum(X0)
X0= X0/Sum
#print sorted(X0)
'''
Sets=OrderSets()
Sum=0
for i in range(len(X0)):
	if X0[i] < 0.00001:
		Z=0
#	        print(i+1,Sets[i],0)
	else:
		prob=float(format(X0.item(i),'.5f'))
                print prob 
                Sum=Sum+prob
#               print Sum
#		print(i+1,Sets[i],prob)
print Sum
F=np.zeros((32,32))#,np.dtype('float64'))
#print 
List=[i for i in range(16,32)]
List1=[i for i in range(48,64)]
List=List+List1
print len(List)
for i in range(len(List)):
    for j in range(len(List)):
        F[i,j]=C[List[i],List[j]]*X0[List[j]]-C[List[j],List[i]]*X0[List[i]]
        


#print F
printZeroIndex(F,32)
#print Sum
#print max(y)
#X=C*X0
#print distance(X0,Xvi )
#print X
#print max(y)
#print X
#print B*X
#print max(y)

#print B
#print A
'''


#printZeroIndex(W,64)


0.0
0.0


"\nSets=OrderSets()\nSum=0\nfor i in range(len(X0)):\n\tif X0[i] < 0.00001:\n\t\tZ=0\n#\t        print(i+1,Sets[i],0)\n\telse:\n\t\tprob=float(format(X0.item(i),'.5f'))\n                print prob \n                Sum=Sum+prob\n#               print Sum\n#\t\tprint(i+1,Sets[i],prob)\nprint Sum\nF=np.zeros((32,32))#,np.dtype('float64'))\n#print \nList=[i for i in range(16,32)]\nList1=[i for i in range(48,64)]\nList=List+List1\nprint len(List)\nfor i in range(len(List)):\n    for j in range(len(List)):\n        F[i,j]=C[List[i],List[j]]*X0[List[j]]-C[List[j],List[i]]*X0[List[i]]\n        \n\n\n#print F\nprintZeroIndex(F,32)\n#print Sum\n#print max(y)\n#X=C*X0\n#print distance(X0,Xvi )\n#print X\n#print max(y)\n#print X\n#print B*X\n#print max(y)\n\n#print B\n#print A\n"

In [3]:
print AllSets()
# Steady state X0 
print X0

[[0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0], [0, 0, 1, 1, 1, 0], [1, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1], [1, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1], [1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1], [0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 1], [1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 0, 1], [0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 1, 1],

In [4]:
#print W.shape[:]
print printZeroIndex(W,64)

0&0.66667&0.66667&0&0.66667&0&0&0&0.66667&0&0&0&0&0&0&0&0.66667&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0.66667&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
1&0&0&0.33333&0&0.83333&0&0&0&0.66667&0&0&0&0&0&0&0&0.83333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
1&0&0&0.33333&0&0&0.66667&0&0&0&0.83333&0&0&0&0&0&0&0&0.83333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
0&\frac{1}{2}&\frac{1}{2}&0&0&0&0&0.83333&0&0&0&0.83333&0&0&0&0&0&0&0&1&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
1&0&0&0&0&0.83333&0.66667&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&0.83333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
0&1&0&0&1&0&0&0.33333&0&0&0&0&0&0.33333&0&0&0&0&0&0&0&1.33333&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
0&0&1&0&1&0&0&\frac{1}{2}&0&0&0&

In [5]:
X1=[]

for i in X0:
    if i < 10**-15:
        X1.append(0)
    else:
        X1.append(float(i))
#Min=min(X1)


In [6]:
print X1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.04779959204136501, 0.04219964002701141, 0.04219964002701142, 0.04599844708727923, 0.042199640027011404, 0.06146359288405984, 0.03480324673099039, 0.066412592333006, 0.042199640027011404, 0.03480324673099039, 0.06146359288405984, 0.066412592333006, 0.04599844708727923, 0.066412592333006, 0.066412592333006, 0.13623040975730805, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.011199819739957336, 0.006466342715128555, 0.006466342715128555, 0.006726617886519654, 0.006466342715128555, 0.0054458774713083535, 0.002467819798806266, 0.005107442738181341, 0.006466342715128557, 0.0024678197988062655, 0.005445877471308352, 0.005107442738181341, 0.006726617886519655, 0.005107442738181341, 0.005107442738181341, 0.010214903490133263]


In [7]:
Sets=OrderSets()
Sum=0
for i in range(len(X0)):
	if X0[i] < 0.00001:
		Z=0
#	        print(i+1,Sets[i],0)
	else:
		prob=float(format(X0.item(i),'.10f'))
                print prob 
                Sum=Sum+prob
#               print Sum
#		print(i+1,Sets[i],prob)
print Sum
F=np.zeros((32,32))#,np.dtype('float64'))
#print 
List=[i for i in range(16,32)]
List1=[i for i in range(48,64)]
List=List+List1
print len(List)
for i in range(len(List)):
    for j in range(len(List)):
        F[i,j]=C[List[i],List[j]]*X0[List[j]]-C[List[j],List[i]]*X0[List[i]]
        


#print F
printZeroIndex(F,32)



0.047799592
0.04219964
0.04219964
0.0459984471
0.04219964
0.0614635929
0.0348032467
0.0664125923
0.04219964
0.0348032467
0.0614635929
0.0664125923
0.0459984471
0.0664125923
0.0664125923
0.1362304098
0.0111998197
0.0064663427
0.0064663427
0.0067266179
0.0064663427
0.0054458775
0.0024678198
0.0051074427
0.0064663427
0.0024678198
0.0054458775
0.0051074427
0.0067266179
0.0051074427
0.0051074427
0.0102149035
0.9999999996
32
0&0.002366806&0.002366806&0&0.002366806&0&0&0&0.002366806&0&0&0&0&0&0&0&-0.0094669145&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
-0.002366806&0&0&0.0018994035&0&0.0051975468&0&0&0&-0.0003629793&0&0&0&0&0&0&0&-0.0043668919&0&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
-0.002366806&0&0&0.0018994035&0&0&-0.0003629793&0&0&0&0.0051975468&0&0&0&0&0&0&0&-0.0043668919&0&0&0&0&0&0&0&0&0&0&0&0&0 \\
0&-0.0018994035&-0.0018994035&0&0&0&0&0.0050814829&0&0&0&0.0050814829&0&0&0&0&0&0&0&-0.0063638609&0&0&0&0&0&0&0&0&0&0&0&0 \\
-0.002366806&0&0&0&0&0.0051975468&-0.0003629793&0&0&0&0&0&0.0018994035&0&0&0&0&0&0&0&

In [8]:
listCurrents =[]
allCurrents=[]
Adj=np.zeros((32,32))#,np.dtype('float64'))
for i in range(32):
    insPosCur=[]
    insNegCur=[]    
    Cur=[]    
    for j in range(32):
        if F[i,j]>0:
            insPosCur.append(j+1)
            Cur.append(j)        
            Adj[i,j]=1
        if F[i,j]<0:
            insNegCur.append(j+1)
            Cur.append(j)         
            Adj[i,j]=1   
        if F[i,j]!=0:
            Adj[i,j]=1  
        else:
            Adj[i,j]=0
    listCurrents.append((insPosCur,insNegCur))
#    print Cur
    print (insPosCur,insNegCur)
    allCurrents.append(Cur)    
print allCurrents

([2, 3, 5, 9], [17])
([4, 6], [1, 10, 18])
([4, 11], [1, 7, 19])
([8, 12], [2, 3, 20])
([6, 13], [1, 7, 21])
([8, 14, 22], [2, 5])
([3, 5, 23], [8, 15])
([7, 16, 24], [4, 6])
([11, 13], [1, 10, 25])
([2, 9, 26], [12, 14])
([12, 15, 27], [3, 9])
([10, 16, 28], [4, 11])
([14, 15], [5, 9, 29])
([10, 16, 30], [6, 13])
([7, 16, 31], [11, 13])
([32], [8, 12, 14, 15])
([1], [18, 19, 21, 25])
([2, 17], [20, 22, 26])
([3, 17], [20, 23, 27])
([4, 18, 19], [24, 28])
([5, 17], [22, 23, 29])
([18, 21], [6, 24, 30])
([19, 21], [7, 24, 31])
([20, 22, 23], [8, 32])
([9, 17], [26, 27, 29])
([18, 25], [10, 28, 30])
([19, 25], [11, 28, 31])
([20, 26, 27], [12, 32])
([13, 21, 25], [30, 31])
([22, 26, 29], [14, 32])
([23, 27, 29], [15, 32])
([24, 28, 30, 31], [16])
[[1, 2, 4, 8, 16], [0, 3, 5, 9, 17], [0, 3, 6, 10, 18], [1, 2, 7, 11, 19], [0, 5, 6, 12, 20], [1, 4, 7, 13, 21], [2, 4, 7, 14, 22], [3, 5, 6, 15, 23], [0, 9, 10, 12, 24], [1, 8, 11, 13, 25], [2, 8, 11, 14, 26], [3, 9, 10, 15, 27], [4, 8, 13, 14,

In [9]:
print listCurrents[20]
print listCurrents[16]
print listCurrents[4]

([5, 17], [22, 23, 29])
([1], [18, 19, 21, 25])
([6, 13], [1, 7, 21])


In [10]:
Sum=0
#print F[20,4]
#print F[20,16]
#print F[20,21]
#print F[20,22]
#print F[20,28]
Sum=Sum+F[4,5]
Sum=Sum+F[4,6]-0.00028
Sum=Sum+F[4,0]
Sum=Sum+F[4,12]
Sum=Sum+F[4,20]
#print Sum
print F[15,11]


-0.0017026125


In [11]:
### Make Directional Graph Straight # 
def make_Straight(F):
    F_Direct=F.copy()
    (x,y)=F.shape[:]
    for i in range(x):
        for j in range(y):
            if F[i,j]>=0:
                F_Direct[j,i]=F[i,j]
            else:
                F_Direct[j,i]=0
    return F_Direct 
F_Direct=make_Straight(F)
np.savetxt('Igraph_Adj.dat', F_Direct, delimiter=',',fmt='%lf')  

In [12]:
#V13+V29+V25+V9
#V19+V17+V18+V20
#print F_Direct
print F_Direct[16,20]
x=F[12,28]+F[28,24]+F[24,8]+F[8,12]
x=F[18,16]+F[16,17]+F[17,19]+F[19,18]
print x

0.0023667385
0.0


In [13]:
print F[1,17]
print F[2,18]
print F[3,19]
print F[4,20]

-0.0043668919
-0.0043668919
-0.0063638609
-0.0043668919


In [14]:
print F[16,0]
print F[16,24]
print F[16,20]

0.0094669145
-0.0023667385
-0.0023667385


In [15]:
## Cutting a subsection of matrix ## 
def Select_Section(List, F):
    Ln=len(List)
    print Ln
    subF=np.zeros((Ln,Ln))
    for i in range(len(List)):
        for j in range(len(List)):
            iIndex=List[i]-1
            jIndex=List[j]-1            
            subF[i,j]=F[iIndex,jIndex]
    return make_Straight(subF)            

In [16]:
List=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]#18,19,20,21,22,23,24]
subF=Select_Section(List, F)
np.savetxt('Igraph_subF_Cube1-2.dat', subF, delimiter=',',fmt='%lf')  

16


In [17]:
List=[3,7,9,11,13,15,27,31]#,8,9,10,11,12,13,14,15,16]#18,19,20,21,22,23,24]
subF=Select_Section(List, F)
np.savetxt('Igraph_subF_CubeInt.dat', subF, delimiter=',',fmt='%lf')  

8


In [18]:
F

array([[ 0.        ,  0.00236681,  0.00236681, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00236681,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00236681,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00170251],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00170251],
       [ 0.        ,  0.        ,  0.        , ...,  0.00170251,
         0.00170251,  0.        ]])

In [19]:
np.savetxt('TranMatrix.dat', W.T, delimiter='\t',fmt='%5f')  

In [20]:
#redM is reduced of W.T
redM=np.zeros((32,32))
def reduceMatrix(W):
    redM=np.zeros((32,32))
    List=[i for i in range(16,32)]
    List1=[i for i in range(48,64)]
    List=List+List1
    for i in range(len(List)):
        for j in range(len(List)):
            redM[i,j]=W[List[j],List[i]]#C[List[i],List[j]]*X0[List[j]]-C[List[j],List[i]]*X0[List[i]]
    return redM


In [21]:
redM=reduceMatrix(W)

In [22]:
for i in range(32):
    insPosCur=[]
    TranProbCur=[]    
    Cur=[]    
    for j in range(32):
        if redM[i,j]>0:
            insPosCur.append(j+1)
            TranProbCur.append(redM[i,j])
    listCurrents.append((insPosCur,TranProbCur))
#    print Cur
    print (i+1,insPosCur,TranProbCur)
    allCurrents.append(Cur)    
#print allCurrents

(1, [2, 3, 5, 9, 17], [1.0, 1.0, 1.0, 1.0, 2.0])
(2, [1, 4, 6, 10, 18], [0.83333, 0.5, 1.0, 1.0, 1.5])
(3, [1, 4, 7, 11, 19], [0.83333, 0.5, 1.0, 1.0, 1.5])
(4, [2, 3, 8, 12, 20], [0.5, 0.5, 1.0, 1.0, 1.33333])
(5, [1, 6, 7, 13, 21], [0.83333, 1.0, 1.0, 0.5, 1.5])
(6, [2, 5, 6, 8, 14, 22], [1.33333, 1.33333, 1.0, 0.5, 0.5, 1.0])
(7, [3, 5, 8, 15, 23], [0.83333, 0.83333, 0.5, 0.5, 1.0])
(8, [4, 6, 7, 8, 16, 24], [1.33333, 0.5, 1.0, 1.0, 0.5, 0.83333])
(9, [1, 10, 11, 13, 25], [0.83333, 1.0, 1.0, 0.5, 1.5])
(10, [2, 9, 12, 14, 26], [0.83333, 0.83333, 0.5, 0.5, 1.0])
(11, [3, 9, 11, 12, 15, 27], [1.33333, 1.33333, 1.0, 0.5, 0.5, 1.0])
(12, [4, 10, 11, 12, 16, 28], [1.33333, 1.0, 0.5, 1.0, 0.5, 0.83333])
(13, [5, 9, 14, 15, 29], [0.5, 0.5, 1.0, 1.0, 1.33333])
(14, [6, 10, 13, 14, 16, 30], [0.5, 1.0, 1.33333, 1.0, 0.5, 0.83333])
(15, [7, 11, 13, 15, 16, 31], [1.0, 0.5, 1.33333, 1.0, 0.5, 0.83333])
(16, [8, 12, 14, 15, 16, 32], [1.0, 1.0, 1.0, 1.0, 2.0, 0.66667])
(17, [1, 18, 19, 21, 25], [0

In [23]:
Z=W.T
redM[0,16]

2.0

In [24]:
X0

matrix([[6.17087696e-18],
        [2.17346687e-18],
        [2.17346687e-18],
        [1.94792211e-18],
        [2.17346687e-18],
        [2.61085868e-18],
        [2.17329319e-18],
        [8.26905618e-19],
        [2.17346687e-18],
        [2.17329319e-18],
        [2.61085868e-18],
        [8.26905618e-19],
        [1.94792211e-18],
        [8.26905618e-19],
        [8.26905618e-19],
        [9.57397820e-19],
        [4.77995920e-02],
        [4.21996400e-02],
        [4.21996400e-02],
        [4.59984471e-02],
        [4.21996400e-02],
        [6.14635929e-02],
        [3.48032467e-02],
        [6.64125923e-02],
        [4.21996400e-02],
        [3.48032467e-02],
        [6.14635929e-02],
        [6.64125923e-02],
        [4.59984471e-02],
        [6.64125923e-02],
        [6.64125923e-02],
        [1.36230410e-01],
        [1.10901007e-18],
        [7.47813020e-19],
        [7.47813020e-19],
        [2.56285106e-19],
        [7.47813020e-19],
        [1.67414143e-19],
        [1.1

In [25]:
F.shape[:]

(32, 32)

In [26]:
## Defining the states ## 
redStates=[]
sX0=[0]
sX1=[1,2,4,8]
sX2=[3,5,6,9,10,12]
sX3=[7,11,13,14]
sX4=[15]
sIX0=[i+16 for i in sX0]
sIX1=[i+16 for i in sX1]
sIX2=[i+16 for i in sX2]
sIX3=[i+16 for i in sX3]
sIX4=[i+16 for i in sX4]

redStates.append(sX0)
redStates.append(sX1)
redStates.append(sX2)
redStates.append(sX3)
redStates.append(sX4)
redStates.append(sIX0)
redStates.append(sIX1)
redStates.append(sIX2)
redStates.append(sIX3)
redStates.append(sIX4)
print redStates

[[0], [1, 2, 4, 8], [3, 5, 6, 9, 10, 12], [7, 11, 13, 14], [15], [16], [17, 18, 20, 24], [19, 21, 22, 25, 26, 28], [23, 27, 29, 30], [31]]


In [27]:
def returnState(index,states):
    for i in range(len(states)):
        for j in range(len(states[i])):
            if index==states[i][j]:
                return i 


In [28]:
redF=np.zeros((10,10))
P0=np.zeros
for i in range(32):
    for j in range(32):
        indexI=returnState(i,redStates) 
        indexJ=returnState(j,redStates)
        redF[indexI,indexJ]+=F[i,j]

In [29]:
redF

array([[ 0.        ,  0.00946722,  0.        ,  0.        ,  0.        ,
        -0.00946691,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.00946722,  0.        ,  0.02693588,  0.        ,  0.        ,
         0.        , -0.01746757,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.02693588,  0.        ,  0.02383613,  0.        ,
         0.        ,  0.        ,  0.00309967,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02383613,  0.        ,  0.00681045,
         0.        ,  0.        ,  0.        ,  0.01702474,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.00681045,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.00680997],
       [ 0.00946691,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.00946695,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.01746757,  0.        ,  0.        ,  0.        ,
         0.00946695,  0.        , -0.02693461

In [30]:
P0=[]
redP0=[0]*10
for i in range(len(X0)):
    if i in List:
        P0.append(float(X0[i]))
for i in range(len(P0)):
    index=returnState(i,redStates)
    redP0[index]+=P0[i]

In [31]:
np.savetxt('redF.dat', redF, delimiter=',',fmt='%5f') 

In [32]:
redF[0,9]

0.0

In [33]:
redF_staight=make_Straight(redF)
np.savetxt('redF.dat', redF_staight, delimiter=',',fmt='%5f') 

In [34]:
redP0

[1.9479221139033502e-18,
 3.82710442640138e-18,
 0.066412592333006,
 0.13623040975730805,
 0,
 0,
 0,
 0,
 0,
 0]

In [35]:
X0

matrix([[6.17087696e-18],
        [2.17346687e-18],
        [2.17346687e-18],
        [1.94792211e-18],
        [2.17346687e-18],
        [2.61085868e-18],
        [2.17329319e-18],
        [8.26905618e-19],
        [2.17346687e-18],
        [2.17329319e-18],
        [2.61085868e-18],
        [8.26905618e-19],
        [1.94792211e-18],
        [8.26905618e-19],
        [8.26905618e-19],
        [9.57397820e-19],
        [4.77995920e-02],
        [4.21996400e-02],
        [4.21996400e-02],
        [4.59984471e-02],
        [4.21996400e-02],
        [6.14635929e-02],
        [3.48032467e-02],
        [6.64125923e-02],
        [4.21996400e-02],
        [3.48032467e-02],
        [6.14635929e-02],
        [6.64125923e-02],
        [4.59984471e-02],
        [6.64125923e-02],
        [6.64125923e-02],
        [1.36230410e-01],
        [1.10901007e-18],
        [7.47813020e-19],
        [7.47813020e-19],
        [2.56285106e-19],
        [7.47813020e-19],
        [1.67414143e-19],
        [1.1

In [47]:
XP=[]
List=[i for i in range(16,32)]
List=List+[i for i in range(48,64)]
print List
for i in List:
    XP.append(float(X0[i])) 

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]


In [49]:
XP

[0.04779959204136501,
 0.04219964002701141,
 0.04219964002701142,
 0.04599844708727923,
 0.042199640027011404,
 0.06146359288405984,
 0.03480324673099039,
 0.066412592333006,
 0.042199640027011404,
 0.03480324673099039,
 0.06146359288405984,
 0.066412592333006,
 0.04599844708727923,
 0.066412592333006,
 0.066412592333006,
 0.13623040975730805,
 0.011199819739957336,
 0.006466342715128555,
 0.006466342715128555,
 0.006726617886519654,
 0.006466342715128555,
 0.0054458774713083535,
 0.002467819798806266,
 0.005107442738181341,
 0.006466342715128557,
 0.0024678197988062655,
 0.005445877471308352,
 0.005107442738181341,
 0.006726617886519655,
 0.005107442738181341,
 0.005107442738181341,
 0.010214903490133263]

In [52]:
XP[31]/XP[15]

0.0749825498457424

In [53]:
XP[30]/XP[14]

0.07690473385787447

In [54]:
reducedStates=[[0],[1,2,4,8],[3,12],[5,10],[6,9],[11,7,13,14],[15],[16],[17,18,20,24],[19,28],[21,26],[22,25],[27,23,29,30],[31]]

In [56]:
XZ=[]
for i in range(len(reducedStates)):
    XZ.append(XP[reducedStates[i][0]])

In [57]:
XZ

[0.04779959204136501,
 0.04219964002701141,
 0.04599844708727923,
 0.06146359288405984,
 0.03480324673099039,
 0.066412592333006,
 0.13623040975730805,
 0.011199819739957336,
 0.006466342715128555,
 0.006726617886519654,
 0.0054458774713083535,
 0.002467819798806266,
 0.005107442738181341,
 0.010214903490133263]

In [69]:
Z1small=XZ[9]-XZ[8]
Z2small=XZ[10]-XZ[12]

In [70]:
for i in XZ:
    print i/Z2small

141.237253043
124.690629821
135.915266918
181.611362156
102.835918788
196.234564105
402.530817386
33.0929973897
19.1066166743
19.8756724062
16.0913669262
7.29186326712
15.0913669262
30.1827870791


In [71]:
for i in XZ:
    print i/Z1small

183.65021829
162.13471228
176.730061655
236.148506043
133.717121556
255.162995295
523.409163576
43.0306881755
24.8442549497
25.8442549497
20.9235381239
9.48157976658
19.6232422435
39.2465536975


In [74]:
for i in XZ:
    print i/((Z2small-Z1small))

611.564228096
539.91653855
588.519767328
786.386099397
445.284568619
849.705280557
1742.9781684
143.294300674
82.7325866665
86.0626356791
69.6764074715
31.5741253405
65.3463584588
130.692947392


In [122]:
import sympy as sy
sState=np.zeros((32,1)) # Fractional steady state # 
sState=sState.astype('object')

symStates={0:[0],1:[1,2,4,8],2:[3,12],3:[5,10],4:[6,9],5:[7,11,13,14],6:[15],7:[16],8:[17,18,20,24],9:[19,28],10:[21,26],11:[22,25],12:[23,27,29,30],13:[31]}
stProb=[]

## Solutions from Mathematica, k=1/2, K=3 ## 
stProb.append(sy.Rational('1157661/24423596'))
stProb.append(sy.Rational('512049/12211798'))
stProb.append(sy.Rational('1123227/24423596'))
stProb.append(sy.Rational('1494801/24423596'))
stProb.append(sy.Rational('1705425/48847192'))
stProb.append(sy.Rational('1625409/24423596'))
stProb.append(sy.Rational('1669143/12211798'))
stProb.append(sy.Rational('133563/12211798'))
stProb.append(sy.Rational('148365/24423596'))
stProb.append(sy.Rational('81747/12211798'))
stProb.append(sy.Rational('64033/12211798'))
stProb.append(sy.Rational('78611/24423596'))
stProb.append(sy.Rational('65601/12211798'))
stProb.append(sy.Rational('65601/6105899'))

for i in range(14):
    for j in symStates[i]:
        sState[j]=stProb[i]

array([512049/6105899], dtype=object)

In [116]:
len(stProb)

14